In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import lightgbm as lgb
import xgboost as xgb

# Load the Titanic datasets
train_df = pd.read_csv('C:/Users/HP PROBOOK/Downloads/XGBM & LGBM/XGBM & LGBM/Titanic_train.csv')
test_df = pd.read_csv('C:/Users/HP PROBOOK/Downloads/XGBM & LGBM/XGBM & LGBM/Titanic_test.csv')

# Preprocessing the 'Name' column
train_df['Title'] = train_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test_df['Title'] = test_df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# Group rare titles into a common category
train_df['Title'] = train_df['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
test_df['Title'] = test_df['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

# Map titles to numeric values
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
train_df['Title'] = train_df['Title'].map(title_mapping)
test_df['Title'] = test_df['Title'].map(title_mapping)

# Drop the 'Name' column
train_df.drop(['Name'], axis=1, inplace=True)
test_df.drop(['Name'], axis=1, inplace=True)

# Drop the 'Ticket' and 'Cabin' columns
train_df.drop(['Ticket', 'Cabin'], axis=1, inplace=True)
test_df.drop(['Ticket', 'Cabin'], axis=1, inplace=True)

# Impute missing values in 'Age' column
imputer = SimpleImputer(strategy='mean')
train_df['Age'] = imputer.fit_transform(train_df[['Age']])
test_df['Age'] = imputer.transform(test_df[['Age']])

# Encode categorical variable 'Sex'
label_encoder = LabelEncoder()
train_df['Sex'] = label_encoder.fit_transform(train_df['Sex'])
test_df['Sex'] = label_encoder.transform(test_df['Sex'])

# Impute missing values in 'Embarked' column
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
test_df['Embarked'].fillna(test_df['Embarked'].mode()[0], inplace=True)

# One-Hot Encoding for 'Embarked' column
onehot_encoder = OneHotEncoder(drop='first', sparse=False)
encoded_features = onehot_encoder.fit_transform(train_df[['Embarked']])
train_df_encoded = pd.concat([train_df, pd.DataFrame(encoded_features, columns=onehot_encoder.get_feature_names_out(['Embarked']))], axis=1)
train_df = train_df_encoded.drop(columns=['Embarked'])

encoded_features = onehot_encoder.transform(test_df[['Embarked']])
test_df_encoded = pd.concat([test_df, pd.DataFrame(encoded_features, columns=onehot_encoder.get_feature_names_out(['Embarked']))], axis=1)
test_df = test_df_encoded.drop(columns=['Embarked'])

# Building Predictive Models

# Split the preprocessed training dataset into training and validation sets
X_train = train_df.drop(columns=['Survived'])
y_train = train_df['Survived']
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Build predictive models using LightGBM and XGBoost algorithms

# LightGBM model
lgb_model = lgb.LGBMClassifier()
lgb_model.fit(X_train_split, y_train_split)
lgb_pred = lgb_model.predict(X_val_split)

# XGBoost model
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train_split, y_train_split)
xgb_pred = xgb_model.predict(X_val_split)

# Evaluate model performance
lgb_accuracy = accuracy_score(y_val_split, lgb_pred)
lgb_precision = precision_score(y_val_split, lgb_pred)
lgb_recall = recall_score(y_val_split, lgb_pred)
lgb_f1 = f1_score(y_val_split, lgb_pred)

xgb_accuracy = accuracy_score(y_val_split, xgb_pred)
xgb_precision = precision_score(y_val_split, xgb_pred)
xgb_recall = recall_score(y_val_split, xgb_pred)
xgb_f1 = f1_score(y_val_split, xgb_pred)

# Print the evaluation metrics
print("LightGBM Metrics:")
print(f"Accuracy: {lgb_accuracy:.2f}")
print(f"Precision: {lgb_precision:.2f}")
print(f"Recall: {lgb_recall:.2f}")
print(f"F1-score: {lgb_f1:.2f}")

print("\nXGBoost Metrics:")
print(f"Accuracy: {xgb_accuracy:.2f}")
print(f"Precision: {xgb_precision:.2f}")
print(f"Recall: {xgb_recall:.2f}")
print(f"F1-score: {xgb_f1:.2f}")

# Comparative Analysis
# Comparing performance metrics of LightGBM and XGBoost models
metrics_df = pd.DataFrame({
    'Model': ['LightGBM', 'XGBoost'],
    'Accuracy': [lgb_accuracy, xgb_accuracy],
    'Precision': [lgb_precision, xgb_precision],
    'Recall': [lgb_recall, xgb_recall],
    'F1-score': [lgb_f1, xgb_f1]
})

print("\nComparative Analysis:")
print(metrics_df)

# Visualize and interpret the results to identify the strengths and weaknesses of each algorithm
metrics_df.set_index('Model').plot(kind='bar', figsize=(10, 6))
plt.title('Comparison of Model Performance Metrics')
plt.xlabel('Model')
plt.ylabel('Score')
plt.xticks(rotation=0)
plt.legend(title='Metrics')
plt.show()

# A brief report summarizing the comparative analysis results and practical implications
print("\nSummary Report:")
print("Both LightGBM and XGBoost models were trained and evaluated on the Titanic dataset.")
print("Comparing their performance metrics, we observe that:")
print("- LightGBM achieved higher accuracy, precision, and F1-score compared to XGBoost.")
print("- XGBoost had slightly higher recall compared to LightGBM.")
print("Overall, LightGBM outperformed XGBoost in terms of predictive performance on this dataset.")
print("However, it's essential to consider other factors such as training time, interpretability, and model complexity "
      "when choosing between these algorithms for practical applications.")